In [2]:
#importing libraries
import os
import cv2
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image

In [5]:
#loading the model weights
model_emotion = model_from_json(open("model.json", "r").read())
model_emotion.load_weights('model.h5')

#haar cascade file
haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')


In [12]:
vid=cv2.VideoCapture(0)
while vid.isOpened():
    _,rec_img=vid.read()
    grayscale_image= cv2.cvtColor(rec_img, cv2.COLOR_BGR2GRAY) #convert rgb to grayscale
    #cv2.imshow('window',grayscale_image)
    faces = haar_cascade.detectMultiScale(grayscale_image,1.1,4)
    for (x,y,w,h) in faces:
       frame = cv2.rectangle(rec_img,(x,y), (x+w,y+h), (255,0,0),thickness=3)
       roi_gray=grayscale_image[y:y+w,x:x+h]
       roi_gray=cv2.resize(roi_gray,(48,48))
       image_pixels = image.img_to_array(roi_gray)
       image_pixels = np.expand_dims(image_pixels, axis = 0)
    
       predictions = model_emotion.predict(image_pixels)
       max_index = np.argmax(predictions[0])
    
       emotion_detection = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
       emotion_prediction = emotion_detection[max_index]
    
       cv2.putText(rec_img, emotion_prediction, (int(x), int(y)),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
       #cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
                   
       resized_image = cv2.resize(rec_img, (1000, 700))
       cv2.imshow('Emotion',resized_image)
    if cv2.waitKey(1) == ord('b'):
         break
vid.release()
cv2.destroyAllWindows()                  